#### 라이브러리 불러오기 

In [ ]:
import pandas as pd
pd.set_option("display.max_row", 100)
pd.set_option("display.max_column", 100)
import numpy as np
import os

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error as mse
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold
from lightgbm import LGBMRegressor

import gower # 거리 계산
import matplotlib.pyplot as plt # 시각화
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram # 클러스터링

import datetime 
from pykrx import stock
import requests
import time

#### 데이터 불러오기 

In [ ]:
cus = pd.read_csv("C:/Users/ambers/Desktop/NH/데이터/open/cus_info.csv")
cus_ = pd.read_csv("C:/Users/ambers/Desktop/NH/데이터/open/re_cus_info.csv")

iem = pd.read_csv("C:/Users/ambers/Desktop/NH/데이터/open/iem_info_20210902.csv")
hist = pd.read_csv("C:/Users/ambers/Desktop/NH/데이터/open/stk_bnc_hist.csv")
train = pd.read_csv("C:/Users/ambers/Desktop/NH/데이터/open/stk_hld_train.csv")
test = pd.read_csv("C:/Users/ambers/Desktop/NH/데이터/open/stk_hld_test.csv")

submission = pd.read_csv("C:/Users/ambers/Desktop/NH/데이터/open/sample_submission.csv")

#### 투자성향 재정의 
##### 투자성향은 개인이 선택해놓은 항목으로 성별, 수익률, 라이프 스테이지 등을 사용하여 재정의 
##### 수익률은 해당 일의 최고가와 최저가의 평균값을 이용하여 매수/매도일의 주가로 계산 

In [ ]:
# 수익률 
import FinanceDataReader as fdr
import math 
import multiprocessing

def bank_id(x,y):
    start=x
    iem_=y[1:]
    c=fdr.DataReader(symbol=iem_,start=str(start),end=str(start))
    if len(c)==0:
        return 0
    return int((c.High+c.Low)/2)

In [ ]:
hist['close']=hist.apply(lambda x: bank_id(x['bse_dt'],x['iem_cd']),axis=1)

In [ ]:
# 수익률 계산 
def income(i):
    if hist.iloc[i-1,1]==hist.iloc[i,1] and hist.iloc[i-1,3]==hist.iloc[i,3] and hist.iloc[i-1,4]>hist.iloc[i,4]:
        hist.iloc[i,8]=(hist.iloc[i,7]*hist.iloc[i-1,4])/(hist.iloc[i-1,7]*hist.iloc[i-1,4])*100-100
        # print(bb.iloc[i,8])
    else: 
        hist.iloc[i,8]=0

In [ ]:
hist['income']=0

if __name__=='__main__':
    # process_list=np.arange(1, len(bb)-1)
    process_list = np.arange(929761,len(bb)-1)

    for _ in process_list:
        income(_)
        t_income(_)
        print(_)
        if _ == len(bb)-1:
            break

    p = multiprocessing.Pool(processes=4) 
    p.map(income,process_list) 
    p.close() 
    p.join()

In [ ]:
train_sort=hist.sort_values(by=['bse_dt'])
day=train_sort['bse_dt'].unique()

In [ ]:
# per 불러오기 
def get_per(date):
    tic=stock.get_market_ticker_list(date,market='ALL')
    
    corp=[]
    
    for ticker in tic:
        corp_name=stock.get_market_ticker_name(ticker)
        corp.append([ticker,corp_name])
        
    df=pd.DataFrame(data=corp, columns=['종목코드','종목명'])
    df=df.set_index('종목코드')
    
    df_f=stock.get_market_fundamental_by_ticker(date,market='ALL')
    
    df=pd.merge(df,df_f,left_index=True, right_index=True)
    
    return df 

In [ ]:
hist['per']=0

def perr(j): # 0-1226 존재 
    t1=train_sort[train_sort['bse_dt']==day[j]]
    per1=get_per(day[j])
    per_iem=list(t1['iem_cd']) # per_iem[1][1:]
    print(j,len(per_iem))
    for i in np.arange(len(per_iem)-1):
        if len(per1[per1.index==per_iem[i][1:]])!=0:
            l=len(hist[(hist['bse_dt']==day[j]) & (hist['iem_cd']==per_iem[i])]['per'])
            hist['per']=np.where((hist['bse_dt']==day[j]) & (hist['iem_cd']==per_iem[i]),
                                    float(per1[per1.index==per_iem[i][1:]]['PER']),hist['per'])
            print(i)


if __name__=='__main__':
    process_list = np.arange(len(day)-1)

    for _ in process_list:
        perr(_)
        if _ == len(day)-1:
            break

    p = multiprocessing.Pool(processes=13) # cpu 개수 16
    p.map(perr,process_list) 
    p.close() 
    p.join()

In [ ]:
# 고객별 수익률을 기하평균을 사용하여 gower거리를 이용한 클러스터링 
from functools import reduce 
import math

def multiply(arr):
    try:
        return math.pow(reduce(lambda x, y : x*y, arr),1/len(arr)) # 근데 누적 수익률은 많이 할 수록 높아지는 거라서 딱히 
    except ValueError:  # Square root of a negative number.
        print("minus")

In [ ]:
income0=hist[hist['income']!=0]
cust_income=income0[['act_id','income']].groupby('act_id').agg(multiply)

In [ ]:
cust_income=cust_income[cust_income.notnull()] # 비상정주식의 주가데이터는 전부 알 수가 없어 구할 수 있는 경우의 고객에 대해서 재정의 

In [ ]:
income_non=cust_income[cust_income['income'].notnull()] # 무한으로 넘어가는 경우가 존재 -> 표준화를 통해 1로 부여
income_non.describe

In [ ]:
income_=income_non[income_non['income']!=np.inf]
income_.describe()

In [ ]:
cust_=pd.DataFrame(income_['income']-np.mean(income_['income']))/np.std(income_['income'])

In [ ]:
income_inf=income_non[income_non['income']==np.inf]
income_inf['income']=1
income_inf

In [ ]:
cust_income=pd.concat([cust_,income_inf])
cust_income

##### 성별, 연령대, 직업군, 수익률, 시가총액을 통해  클러스터링

In [ ]:
cust1=cus.iloc[:,[0,1,2,3,7,4]] # 고객아이디0 성별1, 연령대2, 라이프스테이지6 + 자산구간 4 투자성향=3

In [ ]:
income1=cust_income[['income']]
income1

In [ ]:
tend=pd.merge(cust1,income1,how='left',left_on='act_id',right_on='act_id') # 필요한 데이터셋 
tend.head()

In [ ]:
tend1=tend[tend['income'].notnull()]

In [ ]:
tend_y=tend1['ivs_icn_cd']
tend_x=tend1.iloc[:,1:]

In [ ]:
Xy_original=tend_x.copy()
df=tend_x.copy()

In [ ]:
data_meta = pd.DataFrame(df.nunique(),columns=['num'],index=None).sort_values('num').reset_index()
data_meta.columns = ['name','num']
data_meta['type'] = 'numerical'

In [ ]:
categorical_features = list(tend_x.columns[:4])
numerical_features = list(tend_x.columns[4:])

In [ ]:
cat = [True if x in categorical_features else False for x in Xy_original.columns]

n = 5
focus = Xy_original.sample(1)
similar = gower.gower_topn(focus,Xy_original,
                           n=n,cat_features=cat)

In [ ]:
cat = [True if x in categorical_features else False for x in Xy_original.columns]

%time gd = gower.gower_matrix(Xy_original, cat_features=cat)

In [ ]:
from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from scipy.spatial.distance import pdist,squareform

In [ ]:
# condensed matrix
gdv = squareform(gd,force='tovector')

# output matrix has format [idx1, idx2, dist, sample_count]
Z = linkage(gdv,method='complete')
Z_df = pd.DataFrame(Z,columns=['id1','id2','dist','n'])
Z_df[:2]

In [ ]:
tend1['cluster']=fcluster(Z, 4, criterion='maxclust')
pd.crosstab(tend1['cluster'],tend1['ivs_icn_cd'])

#### train의 hist d
##### ewm을 사용하여 hist_d설정하여 사용 
##### 종목에 따라서 날짜가 가까울수록 놏은 가중치를 부여한 보유 날짜 계산

In [ ]:
train1=pd.merge(train,iem,how='left',left_on='iem_cd',right_on='iem_cd')

In [ ]:
train2=train1.sort_values(by=['stk_dit_cd','byn_dt']) # 시장구분 iem_info stk_dit_cd
train2['hist_d']=train1.sort_values(by=['stk_dit_cd','byn_dt']).groupby(['stk_dit_cd'])['hold_d'].transform(lambda x:x.ewm(span=4,min_periods=1).mean())

In [ ]:
train=train2.iloc[:,:5]

#### LGB

In [ ]:
cus_=pd.merge(cus,cus_[['act_id','cluster']],how='left',left_on='act_id',right_on='act_id')

In [ ]:
cus=cus_.iloc[:,:9]
cus['ivs_icn_cd']=list(cus_['cluster'])

In [ ]:
cus=cus.fillna(5) # 결측치는 새로운 범주값으로 부여 
cus['ivs_icn_cd']=cus['ivs_icn_cd'].astype('category')

In [ ]:
train_data = pd.merge(train, cus, how = "left", on = ["act_id"])
train_data = pd.merge(train_data, iem, how = "left", on = ["iem_cd"])

test_data = pd.merge(test, cus, how = "left", on = ["act_id"])
test_data = pd.merge(test_data, iem, how = "left", on = ["iem_cd"])

train_label = train_data["hold_d"]
train_data.drop(["hold_d"], axis = 1, inplace = True)

In [ ]:
hist["stk_p"] = hist["tot_aet_amt"] / hist["bnc_qty"]
hist = hist.fillna(0)

train_data = pd.merge(train_data, hist, how = "left", on = ["act_id", "iem_cd"])
train_data = train_data[(train_data["byn_dt"] == train_data["bse_dt"])]
train_data.reset_index(drop = True, inplace = True)

test_data = pd.merge(test_data, hist, how = "left", on = ["act_id", "iem_cd"])
test_data = test_data[(test_data["byn_dt"] == test_data["bse_dt"])]
test_data.reset_index(drop = True, inplace = True)

train_data = train_data.drop(["act_id", "iem_cd", "byn_dt", "bse_dt"], axis = 1)
test_data = test_data.drop(["act_id", "iem_cd", "byn_dt", "submit_id", "hold_d", "bse_dt"], axis = 1)

L_encoder = LabelEncoder()
L_encoder.fit(iem["iem_krl_nm"])
train_data["iem_krl_nm"] = L_encoder.transform(train_data["iem_krl_nm"])
test_data["iem_krl_nm"] = L_encoder.transform(test_data["iem_krl_nm"])

In [ ]:
train_data.reset_index(drop = True, inplace=True)
train_label.reset_index(drop = True, inplace=True)

In [ ]:
models = []

folds = KFold(n_splits=10)
for train_idx, val_idx in folds.split(train_data):
    
    train_x = train_data.iloc[train_idx, :]
    train_y = train_label[train_idx]
    val_x = train_data.iloc[val_idx, :]
    val_y = train_label[val_idx]
    
    model = LGBMRegressor(objective= "regression",
                          max_depth= 5,
                          n_estimators= 2000,
                          learning_rate= 0.01,
                          num_leaves = 31)
    
    model.fit(train_x, train_y,
              eval_set=[(val_x, val_y)],
              eval_metric=["rmse"],
              early_stopping_rounds=300,
              verbose=500)
    
    models.append(model)

In [ ]:
result = []
for i in models:
    result.append(i.predict(test_data)) # hist_d 제외시 test_data.iloc[:,1:]
predict = np.mean(result, axis = 0)

In [ ]:
submission["hold_d"] = np.round(predict)

In [ ]:
submission.to_csv("C:/Users/ambers/Desktop/NH/데이터/dacon_ewm1.csv", index = False)